In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[0].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)
print(project_root)

C:\Users\pmasse\Pyprojects\PIM-Recognizer


In [20]:
# imports and customization of diplay
# import os
import re
from functools import partial
from itertools import product
import numpy as np
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from matplotlib import pyplot as plt
import matplotlib.patches as mpatch
import matplotlib.ticker as mtick
import seaborn as sns
# import joblib
import dill
from io import BytesIO

from src.pimest import ContentGetter
from src.pimest import PathGetter
from src.pimest import PDFContentParser
from src.pimest import BlockSplitter
from src.pimest import SimilaritySelector
from src.pimest import custom_accuracy
from src.pimest import text_sim_score
from src.pimest import text_similarity
# from src.pimest import build_text_processor
from src.pimpdf import PDFDecoder

In [19]:
ground_truth_df = pd.read_csv(Path('..') / 'ground_truth' / 'manually_labelled_ground_truth.csv',
                              sep=';',
                              encoding='latin-1',
                              index_col='uid')
ground_truth_uids = list(ground_truth_df.index)

acqui_pipe = Pipeline([('PathGetter', PathGetter(ground_truth_uids=ground_truth_uids,
                                                  train_set_path=Path('..')  / 'ground_truth',
                                                  ground_truth_path=Path('..') / 'ground_truth',
                                                  )),
                        ('ContentGetter', ContentGetter(missing_file='to_nan')),
                        ('ContentParser', PDFContentParser(none_content='to_empty')),
                       ],
                       verbose=True)

texts_df = acqui_pipe.fit_transform(ground_truth_df)
texts_df['ingredients'] = texts_df['ingredients'].fillna('')
texts_df.sample(3)

[Pipeline] ........ (step 1 of 3) Processing PathGetter, total=   0.2s
[Pipeline] ..... (step 2 of 3) Processing ContentGetter, total=   6.4s
Launching 4 processes.
[Pipeline] ..... (step 3 of 3) Processing ContentParser, total= 3.7min


,designation,ingredients,path,content,text
uid,,,,,
23182e34-1f29-4ee2-bbd6-ce820a4b69bc,Taboulé en coupelle 130 g CHARLES FARAUD,"Tomates, eau, BOULGOUR 11,5% (contient du GLUT...",..\ground_truth\23182e34-1f29-4ee2-bbd6-ce820a...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,FICHE TECHNIQUE \n\nPRODUIT FINI\n\n84006787\n...
ec396fb5-89fc-489e-9a39-3d72013227d1,TRIO DE CHAMPIGNONS CULTIVÉS FRANCE,,..\ground_truth\ec396fb5-89fc-489e-9a39-3d7201...,b'%PDF-1.4\n% \xe2\xe3\xcf\xd3\n4\n0\nobj\n<<\...,FICHES TECHNIQUES PRODUITS FINIS SECHES\n\nPRE...
78665d97-164a-4c23-956b-3e41f5f7e423,"Potage indien tandoori en boîte 1,25 kg KNORR","Maltodextrine, amidon modifié de pomme de terr...",..\ground_truth\78665d97-164a-4c23-956b-3e41f5...,b'%PDF-1.5\r%\xe2\xe3\xcf\xd3\r\n337 0 obj\r<<...,Potage Indien Tandoori \n\nKnorr Potage Supé...


In [20]:
# definitions of splitter funcs
splitter_funcs = []
def split_func1(text):
    return(text.split('\n\n'))
splitter_funcs.append(split_func1)
def split_func2(text):
    return(text.split('\n'))
splitter_funcs.append(split_func2)
def split_func3(text):
    regex = r'\s*\n\s*\n\s*'
    return(re.split(regex, text))
splitter_funcs.append(split_func3)

In [21]:
def select_col(df, col_name='blocks'):
        return(df[col_name].fillna(''))
col_selector = FunctionTransformer(select_col)    

In [36]:
process_pipe = Pipeline([('Splitter', BlockSplitter()),
                         ('ColumnSelector', col_selector),
                         ('SimilaritySelector', SimilaritySelector())
                        ],
                       verbose=False)

In [37]:
process_pipe.fit(texts_df, texts_df['ingredients'])

Launching 4 processes.


Pipeline(memory=None,
         steps=[('Splitter',
                 <src.pimest.BlockSplitter object at 0x00000202CC1F8908>),
                ('ColumnSelector',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function select_col at 0x00000202C0B880D8>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, validate=False)),
                ('SimilaritySelector',
                 <src.pimest.SimilaritySelector object at 0x00000202CC1F8088>)],
         verbose=False)

In [40]:
process_pipe.predict(texts_df.sample(4))

Launching 4 processes.


uid
a4de2934-a2c7-4719-8f06-81f41d0447ab    INGREDIENTS : eau, crustacés 24% (coffres et c...
f194c11e-6fec-4f60-919e-758acb0e4dc9                             \nPain Craquant \nEpe...
85bafa77-8b34-4bf4-9db9-eb0a0a56949a    Ail réhydraté (91%) - Origine : Chine. \nSel, ...
9634f1e6-fe7d-4a39-b372-149f4f775fd0    Agriculture Biologique\nCommerce équitable\nGl...
dtype: object

Il me faut un modèle qui prend un binaire et convertit en liste d'ingrédient prédite.

In [ ]:
PDFDecoder

In [44]:
joblib.dump(process_pipe[2], 'similarity_selector.pkl')

['similarity_selector.pkl']

In [45]:
new_model = joblib.load('similarity_selector.pkl')
new_model

In [41]:
def pdf_parser(binary_content):
    return(PDFDecoder.content_to_text(binary_content))

with open(Path('.') / 'pdf_parser.pkl', mode='wb') as file:
    dill.dump(pdf_parser, file)

In [42]:
def split_func(text):
    regex = r'\s*\n\s*\n\s*'
    return(re.split(regex, text))

with open(Path('.') / 'splitter.pkl', mode='wb') as file:
    dill.dump(split_func, file)